<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Daily_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# Imports nécessaires
from langchain_community.document_loaders import WebBaseLoader
from bs4 import SoupStrainer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from typing import TypedDict, List
from langchain.prompts import PromptTemplate
import nest_asyncio
import asyncio

# Activation de nest_asyncio pour Jupyter
nest_asyncio.apply()

# Chargement du document
loader = WebBaseLoader(
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs={"parse_only": SoupStrainer(["title", "h1", "h2", "h3", "p"])}
)
docs = loader.load()
print("Document chargé avec succès!")

# Découpage du document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    add_start_index=True
)
chunks = text_splitter.split_documents(docs)
print(f"Nombre de chunks créés: {len(chunks)}")

# Configuration des embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Création du vectorstore
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)

# Configuration du modèle de langage
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True
)

# Configuration du pipeline
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=128,
        temperature=0.3,
        do_sample=True,
        no_repeat_ngram_size=3,
        device=-1
    )
)

# Définition de l'état RAG
class RAGState(TypedDict):
    question: str
    context: List[str]
    answer: str

# Template du prompt
rag_prompt = PromptTemplate.from_template("""
Based on the following context, provide a clear and concise answer.
If you cannot answer based on the context, say so.

Context: {context}

Question: {question}

Answer:""")

# Fonctions du pipeline
def retrieve(state: RAGState) -> RAGState:
    results = vectorstore.similarity_search(
        state["question"],
        k=3
    )
    state["context"] = [doc.page_content for doc in results]
    return state

# Le début du code reste identique jusqu'à la fonction generate

def generate(state: RAGState) -> RAGState:
    prompt = rag_prompt.format(
        context="\n\n".join(state["context"]),
        question=state["question"]
    )
    # La sortie du modèle est une chaîne de caractères simple
    response = llm(prompt)
    # Nettoyage de la réponse
    if isinstance(response, str):
        state["answer"] = response
    elif isinstance(response, list):
        state["answer"] = response[0]['generated_text']
    else:
        state["answer"] = str(response)
    return state

# Le reste du code reste identique jusqu'aux fonctions de test

async def execute_pipeline():
    print("=== Pipeline Execution ===")

    # Test basique
    question = "D'après l'article, comment les agents IA peuvent-ils être utilisés dans un contexte créatif?"
    try:
        result = app.invoke({
            "question": question,
            "context": [],
            "answer": ""
        })
        print(f"\nQuestion: {question}")
        print(f"Réponse: {result['answer']}")
    except Exception as e:
        print(f"Erreur lors de l'exécution: {str(e)}")

    # Test avec streaming
    print("\nTest de streaming:")
    try:
        async for step in app.astream(
            {
                "question": question,
                "context": [],
                "answer": ""
            },
            stream_mode="updates"
        ):
            print(f"Étape: {list(step.keys())}")
    except Exception as e:
        print(f"Erreur lors du streaming: {str(e)}")

def run_experiments():
    print("\n=== Évaluation et Expérimentation ===")

    # Questions adaptées au contexte
    questions = [
        "Comment les agents IA peuvent-ils améliorer la production créative?",
        "Quels sont les mécanismes de prise de décision des agents IA?",
        "Quels sont les défis dans le développement d'agents IA pour la création de contenu?"
    ]

    for question in questions:
        try:
            result = app.invoke({
                "question": question,
                "context": [],
                "answer": ""
            })
            print(f"\nQuestion: {question}")
            print(f"Réponse: {result['answer']}")
        except Exception as e:
            print(f"Erreur pour la question '{question}': {str(e)}")

    # Test des paramètres de chunking
    print("\nTest des paramètres de découpage:")
    for size in [500, 1000, 1500]:
        overlap = size // 5
        print(f"\nTaille de chunk: {size}, chevauchement: {overlap}")
        try:
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=size,
                chunk_overlap=overlap
            )
            chunks = splitter.split_documents(docs)
            print(f"Nombre de chunks: {len(chunks)}")
        except Exception as e:
            print(f"Erreur avec la taille {size}: {str(e)}")

# Fonction principale avec gestion d'erreurs
async def main():
    try:
        await execute_pipeline()
        run_experiments()
    except Exception as e:
        print(f"Erreur dans l'exécution principale: {str(e)}")

# Pour exécuter dans Jupyter:
await main()



Document chargé avec succès!
Nombre de chunks créés: 37


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== Pipeline Execution ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Erreur lors de l'exécution: string indices must be integers, not 'str'

Test de streaming:
Étape: ['retrieve']


...

Erreur lors du streaming: string indices must be integers, not 'str'

=== Évaluation et Expérimentation ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Erreur pour la question 'Comment les agents IA peuvent-ils améliorer la production créative?': string indices must be integers, not 'str'


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Erreur pour la question 'Quels sont les mécanismes de prise de décision des agents IA?': string indices must be integers, not 'str'
Erreur pour la question 'Quels sont les défis dans le développement d'agents IA pour la création de contenu?': string indices must be integers, not 'str'

Test des paramètres de découpage:

Taille de chunk: 500, chevauchement: 100
Nombre de chunks: 58

Taille de chunk: 1000, chevauchement: 200
Nombre de chunks: 31

Taille de chunk: 1500, chevauchement: 300
Nombre de chunks: 22


In [48]:
# 1. Imports nécessaires
from langchain_community.document_loaders import WebBaseLoader
from bs4 import SoupStrainer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from typing import TypedDict, List
from langchain.prompts import PromptTemplate
import nest_asyncio
import asyncio

# 2. Document Loading
print("=== Document Loading ===")
loader = WebBaseLoader(
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs={
        "parse_only": SoupStrainer(["title", "h1", "h2", "h3", "p"])
    }
)
docs = loader.load()
print(f"Document loaded successfully!")
print(f"Number of documents: {len(docs)}")
print(f"First document preview: {docs[0].page_content[:200]}")

# 3. Document Splitting
print("\n=== Document Splitting ===")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
    add_start_index=True
)
chunks = text_splitter.split_documents(docs)
print(f"Number of chunks created: {len(chunks)}")
print(f"First chunk preview: {chunks[0].page_content[:200]}")

# 4. Vector Store Indexing
print("\n=== Vector Store Indexing ===")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)
print("Vector store created successfully!")

# 5. LangGraph Pipeline Setup
print("\n=== LangGraph Pipeline Setup ===")

class RAGState(TypedDict):
    question: str
    context: List[str]
    answer: str

# Configuration du modèle
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)

llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=128,
        temperature=0.3,
        do_sample=True,
        no_repeat_ngram_size=3,
        device=-1
    )
)

# Prompt template comme spécifié dans l'exercice
rag_prompt = PromptTemplate.from_template("""
Based on the following context, please provide a clear and concise answer.
If the context doesn't contain relevant information, please say so.

Context: {context}

Question: {question}

Answer:""")

def retrieve(state: RAGState) -> RAGState:
    results = vectorstore.similarity_search(
        state["question"],
        k=3
    )
    state["context"] = [doc.page_content for doc in results]
    return state

def generate(state: RAGState) -> RAGState:
    prompt = rag_prompt.format(
        context="\n\n".join(state["context"]),
        question=state["question"]
    )
    response = llm(prompt)
    state["answer"] = response if isinstance(response, str) else response[0]['generated_text']
    return state

# Construction du graph
workflow = StateGraph(RAGState)
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate", generate)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.set_finish_point("generate")

# Compilation
graph = workflow.compile()



=== Document Loading ===
Document loaded successfully!
Number of documents: 1
First document preview: LLM Powered Autonomous Agents | Lil'Log
      LLM Powered Autonomous Agents
    Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demo

=== Document Splitting ===
Number of chunks created: 37
First chunk preview: LLM Powered Autonomous Agents | Lil'Log
      LLM Powered Autonomous Agents

=== Vector Store Indexing ===
Vector store created successfully!

=== LangGraph Pipeline Setup ===


Device set to use cpu


In [49]:
# 6. Pipeline Execution
async def test_pipeline():
    print("\n=== Pipeline Execution ===")

    question = "What are the main components of an AI agent system?"
    print(f"\nTesting basic invoke with question: {question}")

    # Basic invoke
    result = graph.invoke({
        "question": question,
        "context": [],
        "answer": ""
    })
    print(f"Answer: {result['answer']}")

    # Stream updates
    print("\nTesting stream updates:")
    async for step in graph.astream(
        {
            "question": question,
            "context": [],
            "answer": ""
        },
        stream_mode="updates"
    ):
        print(f"Step: {list(step.keys())}")

# 7. Evaluation and Experimentation
def run_experiments():
    print("\n=== Evaluation and Experimentation ===")

    # Test different questions
    test_questions = [
        "What is an AI agent?",
        "How do AI agents make decisions?",
        "What are the key challenges in AI agent development?"
    ]

    for question in test_questions:
        result = graph.invoke({
            "question": question,
            "context": [],
            "answer": ""
        })
        print(f"\nQ: {question}")
        print(f"A: {result['answer']}")

    # Test different chunk parameters
    print("\nTesting chunk parameters:")
    for size in [500, 1000, 1500]:
        overlap = size // 5
        print(f"\nChunk size: {size}, overlap: {overlap}")
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=size,
            chunk_overlap=overlap
        )
        chunks = splitter.split_documents(docs)
        print(f"Number of chunks: {len(chunks)}")

# 8. Main execution
async def main():
    await test_pipeline()
    run_experiments()

# Pour exécuter dans Jupyter:
await main()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== Pipeline Execution ===

Testing basic invoke with question: What are the main components of an AI agent system?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: 
Based on the following context, please provide a clear and concise answer.
If the context doesn't contain relevant information, please say so.

Context: Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.Agent System Overview#In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:Component One: Planning#A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.Task Decomposition#Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to

Building agents with LLM (large language model

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Step: ['generate']

=== Evaluation and Experimentation ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Q: What is an AI agent?
A: 
Based on the following context, please provide a clear and concise answer.
If the context doesn't contain relevant information, please say so.

Context: LLM Powered Autonomous Agents | Lil'Log
      LLM Powered Autonomous Agents

LLM Powered Autonomous Agents | Lil'Log
      LLM Powered Autonomous Agents

LLM Powered Autonomous Agents | Lil'Log
      LLM Powered Autonomous Agents

Question: What is an AI agent?

Answer: An AI agent is a robot that is a human.
Answer.
Question.
What is an autonomous agent? What is a machine? What are the robots?
Answer,
Question,
Answer
Question
Question,
Question (Answer)
Question(Answer)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Q: How do AI agents make decisions?
A: 
Based on the following context, please provide a clear and concise answer.
If the context doesn't contain relevant information, please say so.

Context: refuse to follow an instruction). Consequently, much of the agent demo code focuses on parsing model output.Citation#Cited as:Weng, Lilian. (Jun 2023). “LLM-powered Autonomous Agents”. Lil’Log. https://lilianweng.github.io/posts/2023-06-23-agent/.OrReferences#[1] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022[2] Yao et al. “Tree of Thoughts: Dliberate Problem Solving with Large Language Models.” arXiv preprint arXiv:2305.10601 (2023).[3] Liu et al. “Chain of Hindsight Aligns Language Models with Feedback

refuse to follow an instruction). Consequently, much of the agent demo code focuses on parsing model output.Citation#Cited as:Weng, Lilian. (Jun 2023). “LLM-powered Autonomous Agents”. Lil’Log. https://lilianweng.github.io/posts/2023-06-23-agent